In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [10]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = word_tokenize(text)  # Use word_tokenize for more accurate tokenization
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in word_tokenize(text) if word.lower() not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    return ''.join([char for char in text if not char.isdigit()])

def lower_case(text):
    """Convert text to lower case."""
    text = word_tokenize(text)  # Tokenize before applying lower case
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")  # Additional punctuation you may want to remove
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Appsquadz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Appsquadz\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Appsquadz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Appsquadz\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [11]:
df = normalize_text(df)
df.head()

,sentiment,content
0,empty,tiffanylue know listenin bad habit earlier sta...
1,sadness,layin n bed headache ughhhh waitin call
2,sadness,funeral ceremony gloomy friday
3,enthusiasm,want hang friend soon
4,neutral,dannycastillo want trade someone houston ticke...


In [12]:
df['sentiment'].value_counts()

sentiment
neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: count, dtype: int64

In [13]:
x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

In [14]:
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})
df.head()

C:\Users\Appsquadz\AppData\Local\Temp\ipykernel_23896\1089524538.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})
C:\Users\Appsquadz\AppData\Local\Temp\ipykernel_23896\1089524538.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})


,sentiment,content
1,0,layin n bed headache ughhhh waitin call
2,0,funeral ceremony gloomy friday
6,0,sleep im thinking old friend want s married da...
8,0,charviray charlene love miss
9,0,kelcouch m sorry least s friday


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['sentiment'])

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=2000)
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

In [20]:
import dagshub
import mlflow
mlflow.set_tracking_uri('https://dagshub.com/neerajstd159/mlflow-demo.mlflow')
dagshub.init(repo_owner="neerajstd159", repo_name="mlflow-demo", mlflow=True)

Initialized MLflow to track repo "neerajstd159/mlflow-demo"

Repository neerajstd159/mlflow-demo initialized!

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

In [ ]:
import mlflow.models


mlflow.set_experiment('Logistic Regression Baseline')
with mlflow.start_run():
    mlflow.log_param("vectorizer", "Bag of Words")
    mlflow.log_param("max_features", "2000")
    mlflow.log_param("test_size", 0.2)
    mlflow.log_param("model", "Logistic Regression")

    lr = LogisticRegression()
    lr.fit(X_train_bow, y_train)
    y_pred = lr.predict(X_test_bow)

    accuracy = accuracy_score(y_pred, y_test)
    recall = recall_score(y_pred, y_test)
    f1_scr = f1_score(y_pred, y_test)
    precision =precision_score(y_pred, y_test)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("f1_score", f1_scr)
    signature = mlflow.models.infer_signature(X_train_bow, lr.predict(X_test_bow))
    mlflow.sklearn.log_model(lr, "model", signature=signature)

    # Save and log the notebook
    import os
    notebook_path = "baseline_model.ipynb"
    os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
    mlflow.log_artifact(notebook_path)

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1_scr}")


Accuracy: 0.7825751734772552
Precision: 0.7953488372093023
Recall: 0.7737556561085973
F1 Score: 0.7844036697247706
🏃 View run bedecked-snail-32 at: https://dagshub.com/neerajstd159/mlflow-demo.mlflow/#/experiments/1/runs/ccaa7056db1a439bb6ce469dc37052b5
🧪 View experiment at: https://dagshub.com/neerajstd159/mlflow-demo.mlflow/#/experiments/1
